In [ ]:
import openai
import pandas as pd

# Set your OpenAI API key
# openai.api_key = "your-openai-api-key"
openai.api_key = ""

def refine_transcript_chunk(chunk):
    """
    Refines a single chunk of the transcript using OpenAI API.
    """
    prompt = f"""
    You are tasked with refining a portion of a classroom audio transcription to make it coherent, structured, and accurate.
    Follow these rules:
    1. Consolidate the speech of the same speaker into cohesive blocks while preserving logical flow and context.
    2. Clearly differentiate between teacher and students based on the following:
        - The teacher addresses students by their names when asking questions.
        - Assume the reply immediately following a question is by the addressed student, unless explicitly attributed otherwise.
        - If the teacher uses ambiguous terms (e.g., "um," "Em," "uh," or similar), recognize these as filler words unless clearly referring to a student.
    3. Unless explicitly addressed, assume the speaker is the teacher.
    4. Retain timestamps where available and align them with the refined text.
    5. Organize the transcript clearly with "Teacher" and "Student <Name>" labels, ensuring the flow of dialogue makes sense.
    6. Eliminate filler words like "um," "uh," or other pauses, unless critical to the dialogue or context.
    7. Improve grammar, punctuation, and formatting for clarity and professionalism.

    Here is a portion of the original transcript:
    {chunk}

    Now provide the fully refined and improved portion:
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a skilled text refinement assistant."},
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
            max_tokens=2000,
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"An error occurred while refining a chunk: {e}")
        return None

def split_transcript(transcript, max_tokens=3000):
    """
    Splits the transcript into smaller chunks that fit within the token limit.
    """
    words = transcript.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

def refine_classroom_transcript(input_path, refined_path):
    """
    Refines a long classroom transcript by splitting it into smaller chunks and saving the refined output.
    """
    with open(input_path, "r") as file:
        transcript = file.read()

    chunks = split_transcript(transcript)

    refined_transcript = []
    for i, chunk in enumerate(chunks):
        print(f"Refining chunk {i + 1} of {len(chunks)}...")
        refined_chunk = refine_transcript_chunk(chunk)
        if refined_chunk:
            refined_transcript.append(refined_chunk)
        else:
            print(f"Skipping chunk {i + 1} due to an error.")

    with open(refined_path, "w") as file:
        file.write("\n\n".join(refined_transcript))
    
    print(f"Refined transcript saved to {refined_path}")

def split_for_analysis(transcript, max_tokens=4000):
    """
    Splits the refined transcript into smaller chunks for analysis.

    Args:
    - transcript (str): The refined transcript.
    - max_tokens (int): Maximum token limit for each chunk.

    Returns:
    - list: List of transcript chunks.
    """
    words = transcript.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


def analyze_transcript(file_path):
    """
    Analyze the classroom transcript and generate structured data for students and teachers.

    Args:
    - file_path (str): Path to the transcript text file.

    Returns:
    - str: Generated structured data from OpenAI API.
    """
    try:
        # Load the transcript from the text file
        with open(file_path, "r") as file:
            transcript = file.read()
        # Split refined transcript into chunks
        chunks = split_for_analysis(transcript)
        # Analyze each chunk
        combined_results = []
        for i, chunk in enumerate(chunks):
            print(f"Analyzing chunk {i + 1} of {len(chunks)}...")
            # Define the prompt
            prompt = f"""
            You are tasked with analyzing the classroom transcript provided below to generate detailed and specific learnings and improvements for both students and teachers.

            Here is the transcript:
            {chunk}

            Instructions:
            1. Analyze the transcript and identify positive learnings done by students, mentioning their names and ensure you correctly identify when a student is speaking versus when the teacher is speaking.
            - Use context to differentiate random words or filler terms from actual student names.
            - Only consider a word to be a student name if the teacher directly addresses the person (e.g., "Adrian, can you tell me...") or if it is clearly followed by a student's response.
            2. Highlight improvements needed for students based on their participation or responses.
            3. For teachers:
            - Capture specific positive enhancements (e.g., teaching methods, interactions).
            - Mention improvements needed in their teaching style or interactions.
            4. Output two separate tables:
            - One for students with columns: "Date", "Student Name", "Positive Learnings", "Improvements Needed".
            - One for teachers with columns: "Date", "Teacher Name", "Positive Enhancements", "Improvements Needed".
            5. Analyze the transcript carefully:
            - Identify positive learnings done by students based on their participation or responses, mentioning their names.
            - Highlight improvements needed for students if their responses are incomplete or they were not engaged.
            - Capture specific positive enhancements for teachers (e.g., encouraging participation, providing examples).
            - Highlight improvements needed for teachers (e.g., managing pacing, giving clearer instructions).
            6. Assume today's date for all records and generate realistic and specific data.
            7. Ensure that:
            - "Student Name" is populated only when it is explicitly mentioned in the transcript or contextually clear.
            - Generic filler words like "Em" or random phrases are not misinterpreted as names.
            - The output is realistic and specific.

            Provide the data in a structured format with the following tables:
            - Table 1: Students
            - Table 2: Teachers
            """
            try:
                # Call OpenAI API
                response = openai.ChatCompletion.create(
                    model="gpt-4",
                    messages=[
                        {"role": "system", "content": "You are a skilled data analysis assistant."},
                        {"role": "user", "content": prompt},
                    ],
                    temperature=0.7,
                    max_tokens=2000,
                )
                combined_results.append(response["choices"][0]["message"]["content"])
            except Exception as e:
                print(f"Error analyzing chunk {i + 1}: {e}")
            
            return "\n\n".join(combined_results)

    except FileNotFoundError:
        print(f"Error: File not found at path {file_path}. Please provide a valid file path.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def convert_openai_response_to_excel(response, student_file, teacher_file):
    """
    Converts OpenAI response into two Excel files for students and teachers.

    Args:
    - response (str): The OpenAI response containing structured tables.
    - student_file (str): Path to save the Excel file for student data.
    - teacher_file (str): Path to save the Excel file for teacher data.
    """
    try:
        # Locate "Table 1: Students" and "Table 2: Teachers"
        student_start = response.find("Table 1: Students")
        teacher_start = response.find("Table 2: Teachers")

        if student_start == -1:
            print("Warning: 'Table 1: Students' not found in the response.")
            student_section = None
        else:
            student_section = response[student_start:teacher_start if teacher_start != -1 else len(response)].strip()

        if teacher_start == -1:
            print("Warning: 'Table 2: Teachers' not found in the response.")
            teacher_section = None
        else:
            teacher_section = response[teacher_start:].strip()

        # Process student data
        if student_section:
            student_lines = student_section.split("\n")
            student_columns = student_lines[2].strip("|").split("|")  # Extract column headers
            student_columns = [col.strip() for col in student_columns if col.strip()]
            student_rows = [
                [field.strip() for field in line.strip("|").split("|")]
                for line in student_lines[4:] if line.strip() and "|" in line
            ]
            student_df = pd.DataFrame(student_rows, columns=student_columns)
        else:
            print("Warning: No student data found in the response.")
            student_df = pd.DataFrame(columns=["Date", "Student Name", "Positive Learnings", "Improvements Needed"])

        # Process teacher data
        if teacher_section:
            teacher_lines = teacher_section.split("\n")
            teacher_columns = teacher_lines[2].strip("|").split("|")  # Extract column headers
            teacher_columns = [col.strip() for col in teacher_columns if col.strip()]
            teacher_rows = [
                [field.strip() for field in line.strip("|").split("|")]
                for line in teacher_lines[4:] if line.strip() and "|" in line
            ]
            teacher_df = pd.DataFrame(teacher_rows, columns=teacher_columns)
        else:
            print("Warning: No teacher data found in the response.")
            teacher_df = pd.DataFrame(columns=["Date", "Teacher Name", "Positive Enhancements", "Improvements Needed"])

        # Save to Excel files
        student_df.to_excel(student_file, index=False)
        teacher_df.to_excel(teacher_file, index=False)

        print(f"Excel files created:\n- Students: {student_file}\n- Teachers: {teacher_file}")

    except Exception as e:
        print(f"An error occurred while converting to Excel: {e}")


def automate_pipeline(input_path, refined_path, student_file, teacher_file):
    """
    Automates the pipeline: refining transcript, analyzing it, and saving results to Excel.
    """
    refine_classroom_transcript(input_path, refined_path)
    generated_data = analyze_transcript(refined_path)
    if generated_data:
        convert_openai_response_to_excel(generated_data, student_file, teacher_file)
    else:
        print("Analysis failed. No Excel files generated.")

# Example usage
input_file = "D:/EducareAI/first_task/4th_Grade_Science_binary.txt"
refined_file = "D:/EducareAI/first_task/refined_text/4th_Grade_Science_binary_refined.txt"
student_excel = "D:/EducareAI/first_task/analysis/4th_Grade/students_analysis.xlsx"
teacher_excel = "D:/EducareAI/first_task/analysis/4th_Grade/teachers_analysis.xlsx"

automate_pipeline(input_file, refined_file, student_excel, teacher_excel)


Refining chunk 1 of 8...
Refining chunk 2 of 8...
Refining chunk 3 of 8...
Refining chunk 4 of 8...
Refining chunk 5 of 8...
Refining chunk 6 of 8...
Refining chunk 7 of 8...
Refining chunk 8 of 8...
Refined transcript saved to D:/EducareAI/first_task/refined_text/4th_Grade_Science_binary_refined.txt
Analyzing chunk 1 of 5...
An error occurred while converting to Excel: Response does not contain both student and teacher tables.


In [21]:
generated_data = analyze_transcript(refined_file)
print(generated_data)
# if generated_data:
#         convert_openai_response_to_excel(generated_data, student_excel, teacher_excel)

Analyzing chunk 1 of 5...
Table 1: Students

| Date | Student Name | Positive Learnings | Improvements Needed |
|------|--------------|--------------------|---------------------|
| Today's Date | Unidentified Student | Identified features of an insect and applied this knowledge to assert that a cricket is an insect | Could work on being more confident when sharing their thoughts |
| Today's Date | Isabel | Participated actively in the discussion and was able to share her thoughts | Need to improve on the understanding of insect anatomy, specifically the number of legs |
| Today's Date | Unidentified Student | Disagreed respectfully with Isabel and correctly identified the number of cricket's legs | Can work on providing a more detailed explanation to support their argument |
| Today's Date | Skylar | Confirmed that a cricket is an insect by observing and discussing with a partner | Could improve by providing more reasoning or facts to support their argument |
| Today's Date | Unidentif

In [ ]:


if generated_data:
        convert_openai_response_to_excel_1(generated_data, student_excel, teacher_excel)


Excel files created:
- Students: D:/EducareAI/first_task/analysis/4th_Grade/students_analysis.xlsx
- Teachers: D:/EducareAI/first_task/analysis/4th_Grade/teachers_analysis.xlsx
